In [1]:
from architectural_principles import ArchitecturalConstraints
from train import ExperimentManager

In [6]:
config = {
    'env': {
        'grid_size': (10, 10),
        'max_steps': 500,
        'required_rooms': ArchitecturalConstraints.default_rooms()
    },
    'algorithms': {
        'value_iteration': {
            'gamma': .95,
            'theta': 0.001
        }
    }
}

In [7]:
exm = ExperimentManager(config)

In [ ]:
exm.run_experiments()

In [ ]:
import torch
import numpy as np
import concurrent.futures
from torch.multiprocessing import Pool


def generate_room_dimensions_for_room(room_ranges_items: tuple):
        """
        Generate the dimensions (width, height) combinations for a single room.
        This is a helper function that can be run in parallel.
        """
        room, (w_range, h_range) = room_ranges_items
        widths = np.arange(w_range[0], w_range[1] + 1)
        heights = np.arange(h_range[0], h_range[1] + 1)
        # Create a mesh grid of all width and height combinations
        w_grid, h_grid = np.meshgrid(widths, heights)
        # Stack the grids to get a list of (w, h) tuples
        room_dimensions = np.vstack([w_grid.ravel(), h_grid.ravel()]).T
        return room, room_dimensions


# Check overlap using batched parallel computation
def check_overlaps(positions_batch):
    batch_size = positions_batch.size(0)
    grid = torch.zeros((batch_size, G_width, G_height), dtype=torch.int32, device='cuda')  # Use int32 for counting

    # Iterate over the batch and unpack each room's coordinates (x, y, w, h)
    for i in range(batch_size):
        room = positions_batch[i]  # Now room is a tensor of shape [4] (x, y, w, h)
        
        # Ensure that room is a 1D tensor (x, y, w, h)
        if room.dim() != 1 or room.size(0) != 4:
            print(f"Unexpected room shape: {room.shape}")
            continue

        x, y, w, h = room[0].item(), room[1].item(), room[2].item(), room[3].item()  # Get scalar values

        # Fill grid for each configuration in batch
        for bx in range(x, x + w):
            for by in range(y, y + h):
                grid[i, bx, by] += 1  # Increment cells for each room's area

    # Valid if all grid cells are <= 1
    return (grid <= 1).all(dim=(1, 2))  # Check if no cell is occupied by more than 1 room




def generate_room_configs_parallel(room_ranges: dict, grid_size: tuple):
    G_width, G_height = grid_size

    # Create ranges for each room type
    room_types = list(room_ranges.keys())

    def generate_room_dimensions_parallel(room_ranges):
        """
        Generate room dimensions for all rooms in parallel.
        """
        dimensions = {}        
        # Use ProcessPoolExecutor to parallelize the generation of dimensions
        executor = concurrent.futures.ThreadPoolExecutor()
        for result in executor.map(generate_room_dimensions_for_room, room_ranges.items()):
            room, room_dimensions = result
            dimensions[room] = room_dimensions
        return dimensions
    
    dimensions = generate_room_dimensions_parallel(room_ranges)
    # Generate all possible positions for a room given its dimensions
    def generate_positions(dimensions):
        positions = []
        for w, h in dimensions:
            for x in range(G_width - w + 1):
                for y in range(G_height - h + 1):
                    positions.append((x, y, w, h))
        return positions

    # Generate position tensors for all rooms
    all_positions = [torch.tensor(generate_positions(dimensions[room]), dtype=torch.int32, device='cuda') for room in room_types]
    all_positions_flattend = [pos.flatten() for pos in all_positions]
    product = torch.meshgrid(*all_positions_flattend, indexing='ij')
    all_batches = torch.stack(product, dim=-1).reshape(-1, len(all_positions_flattend))
    print("batched")
    with Pool() as pool:
        results = pool.map(check_overlaps, all_batches)

    # Combine all room positions in parallel, iterate over all possible combinations of room placements
    valid_combinations = []
    
    # Generate all possible combinations of positions for each room
    from itertools import product
    for batch in product(*all_positions):
        batch_tensor = torch.stack(batch, dim=0).to('cuda')  # Stack into a single tensor of shape (num_rooms, num_positions, 4)
        # Check for overlap, ensure the result is True for all rooms in the batch
        if check_overlaps(batch_tensor).all():  # .all() ensures the batch is valid
            valid_combinations.append(batch_tensor.cpu().tolist())
        
    return valid_combinations


# Example usage
room_ranges = {
    "R1": ((2, 4), (2, 4)),
    "R2": ((1, 3), (1, 3)),
#     "R3": ((3, 5), (3, 5)),
#     "R4": ((2, 3), (2, 3)),
#     "R5": ((1, 2), (1, 2)),
#     "R6": ((2, 5), (1, 4))  # Added a 6th room type
}
grid_size = (10, 10)
states = generate_room_configs_parallel(room_ranges, grid_size)
print(f"Total valid configurations: {len(states)}")


batched
